prova
• 25% anàlisis dels atributs (data mining),
• 25% aplicació de diversos mètodes d'aprenentatge,
• 30% visualització i presentació dels resultats,
• 10% repositori github
• 10% presentació opcional

# Cas Kaggle - F1 World Championship (1950 - 2024)
## Introducció
Aquesta base de dades conté informació sobre el campionat del món de F1 des de 1950 fins a 2024. Les dades estan organitzades en diferents arxius CSV dins de la carpeta `data`, cadascun proporciona informació específica sobre diferents aspectes del campionat. A continuació, es descriuen els arxius disponibles:

- `circuits.csv`: Informació sobre els circuits on s'han celebrat curses de F1.
- `constructor_results.csv`: Resultats de cada equip (constructor) en cada cursa.
- `constructor_standings.csv`: Classificació final de cada equip (constructor) al final de cada temporada.
- `constructors.csv`: Informació sobre els equips (constructors) que han participat en temporades de F1.
- `driver_standings.csv`: Classificació final de cada pilot al final de cada temporada.
- `drivers.csv`: Informació sobre els pilots que han participat en temporades de F1.
- `lap_times.csv`: Temps de cada volta de cada cursa.
- `pit_stops.csv`: Informació sobre les parades en 'pit' de cada cursa.
- `qualifying.csv`: Resultats de la sessió de classificació de cada cursa.
- `races.csv`: Informació sobre cada cursa, inclòs dates i ubicacions.
- `results.csv`: Resultats detallats de cada cursa.
- `seasons.csv`: Informació sobre cada temporada del campionat.
- `sprint_results.csv`: Resultats de les curses sprint.
- `status.csv`: Informació sobre l'estat dels pilots durant les curses. (per exemple, si han abandonat o han estat desqualificats).

## Objectius
1. **Anàlisis de rendiment dels pilots i equips**: Evaluar el rendiment dels pilots i equips al llarg dels anys. Identificar els pilots i equips més exitosos de la història de la F1.

2. **Anàlisis de circuits**: Analitzar el rendiment dels pilots i equips en diferents circuits. Identificar els circuits més desafiants i aquells on s'han produït més accidents.

3. **Tendencies històriques**: Identificar tendencies i canvis en el campionat de F1 al llarg del temps. Analitzar l'evolució de la tecnologia i el seu impacte en el rendiment dels equips.

4. **Predicció de resultats**: Utilitzar tècniques d'aprenentatge automàtic per predir els resultats de curses futures. Identificar els factors clau que influeixen en el rendiment dels pilots i equips.